## NAICS table creation
Import and standardize the tables in this folder.  Input files from https://www.census.gov/naics/?48967

In [1]:
import pandas as pd
import numpy as np

## 2002 Data

In [2]:
naics_2002 = pd.pandas.read_fwf('./naics_2002_hier.txt')
naics_2002 = naics_2002.iloc[2:,0:2]
naics_2002.columns = ['NAICS', 'NAICS_desc']
naics_2002['year'] = 2002

In [3]:
naics_2002

,NAICS,NAICS_desc,year
2,11,"Agriculture, Forestry, Fishing and Hunting",2002
3,111,Crop Production,2002
4,1111,Oilseed and Grain Farming,2002
5,11111,Soybean Farming,2002
6,111110,Soybean Farming,2002
...,...,...,...
2338,9281,National Security and International Affairs,2002
2339,92811,National Security,2002
2340,928110,National Security,2002
2341,92812,International Affairs,2002


## 2007-2017 Data

In [4]:
year_list = [2007, 2012, 2017]
file_list = ['./naics_2007_hier.csv', 'naics_2012.csv', 'naics_2017.csv']
file_dict = dict(zip(year_list, file_list))

In [5]:
def file_read(fn):
    try:
        data = pd.read_csv(fn, dtype='str') 
    except:
        data = pd.read_csv(fn, encoding="latin-1", dtype='str') 

    sel_cols = [d for d in data.columns if not('Seq' in d or 'Unnamed' in d)]
    data = data[sel_cols]
    data.columns = ['NAICS', 'NAICS_desc']
    data.dropna(inplace=True)
    return data

In [6]:
naics_later = pd.concat([file_read(f) for f in file_list],
                        keys=year_list) \
    .reset_index(level=0) \
    .rename(columns={'level_0':'year'})

In [7]:
naics_all_long = pd.concat([naics_2002, naics_later])
naics_all_long['year'].value_counts()

year
2002    2341
2007    2328
2012    2209
2017    2196
Name: count, dtype: int64

In [8]:
naics_all_long['NAICS_level'] = naics_all_long['NAICS'].str.len()

In [9]:
naics_all_long['NAICS_level'].value_counts()

NAICS_level
6    4476
5    2880
4    1253
3     397
2      68
Name: count, dtype: int64

In [10]:
pd.crosstab(naics_all_long['year'], naics_all_long['NAICS_level'])

NAICS_level,2,3,4,5,6
year,,,,,
2002,17,100,317,728,1179
2007,17,99,313,724,1175
2012,17,99,312,716,1065
2017,17,99,311,712,1057


##### Identify sectors
Make these level "2"

In [11]:
naics_all_long['sector_flag'] = np.where((naics_all_long['NAICS_level']  == 2) |
                                         naics_all_long['NAICS'].str.contains('-'),
                                        1, 0)
naics_all_long['NAICS_level'] = np.where((naics_all_long['sector_flag']  == 1), 2,
                                       naics_all_long['NAICS_level'])

In [12]:
naics_all_long['sector_flag'].value_counts()

sector_flag
0    8994
1      80
Name: count, dtype: int64

In [13]:
sector_cases= naics_all_long[naics_all_long['sector_flag'] == 1].copy()
sector_cases[['a', 'b']] = sector_cases[sector_cases['sector_flag'] == 1] \
    ['NAICS'].str.split('-', n=2, expand=True)
sector_cases['b'] = sector_cases['b'].fillna(sector_cases['a'])

In [14]:
#sector_cases_long = sector_cases.melt(id_vars=['year', 'NAICS'], value_vars=['a','b'])
sector_cases['sector_range'] = sector_cases \
    .apply(lambda x: [i for i in range(int(x.a),int(x.b)+1)], axis=1) 

In [15]:
sector_cases_long = sector_cases.explode('sector_range') \
    .rename(columns={'NAICS':'NAICS_sector_range',
                    'sector_range':'NAICS'}) \
    .drop(columns=['a', 'b'])

In [16]:
sector_cases_long.sample(4)

,NAICS_sector_range,NAICS_desc,year,NAICS_level,sector_flag,NAICS
1837,62,Health Care and Social Assistance,2017,2,1,62
1,11,"Agriculture, Forestry, Fishing and Hunting",2017,2,1,11
1402,51,Information,2012,2,1,51
274,31-33,Manufacturing,2007,2,1,31


In [17]:
sector_cases_long[sector_cases_long['NAICS_sector_range'] == '31-33'].sample(2)

,NAICS_sector_range,NAICS_desc,year,NAICS_level,sector_flag,NAICS
278,31-33,Manufacturing,2012,2,1,32
274,31-33,Manufacturing,2007,2,1,31


In [18]:
sector_cases_long['NAICS'] = sector_cases_long['NAICS'].astype('str')

In [19]:
naics_all_long = pd.concat([naics_all_long[naics_all_long['sector_flag'] == 0],
                            sector_cases_long]) \
    .sort_values(['year', 'NAICS']) \
    .drop(columns='sector_flag')

In [20]:
naics_all_long.head(3)

,NAICS,NAICS_desc,year,NAICS_level,NAICS_sector_range
2,11,"Agriculture, Forestry, Fishing and Hunting",2002,2,11
3,111,Crop Production,2002,3,NaN
4,1111,Oilseed and Grain Farming,2002,4,NaN


In [21]:
naics_all_long.to_csv('./naics_desc_long.csv', index=False)
naics_all_long.to_parquet('./naics_desc_long.parquet')

## Make wide format version

##### Year of code usage

In [22]:
naics_yearsum = naics_all_long.groupby('NAICS') \
    ['year'] \
    .agg(['min', 'max']) \
    .set_axis(['year_min', 'year_max'], axis=1) \
    .reset_index()

In [23]:
naics_yearsum['year_min'].value_counts()

year_min
2002    2345
2012      84
2017      28
2007       9
Name: count, dtype: int64

In [24]:
naics_yearsum['year_max'].value_counts()

year_max
2017    2200
2007     203
2012      43
2002      20
Name: count, dtype: int64

##### All descriptions

In [25]:
naics_wide = naics_all_long.pivot(index=['NAICS', 'NAICS_level'], columns='year',
                                  values='NAICS_desc')

In [26]:
naics_wide.columns = ['NAICS_desc_' + str(y) for y in naics_wide.columns]

In [27]:
naics_wide = naics_wide.reset_index().merge(naics_yearsum, on='NAICS')

In [28]:
naics_wide[naics_wide['year_max'] == 2007].sample(3)

,NAICS,NAICS_level,NAICS_desc_2002,NAICS_desc_2007,NAICS_desc_2012,NAICS_desc_2017,year_min,year_max
1095,339911,6,Jewelry (except Costume) Manufacturing,Jewelry (except Costume) Manufacturing,NaN,NaN,2002,2007
1318,443111,6,Household Appliance Stores,Household Appliance Stores,NaN,NaN,2002,2007
443,315191,6,Outerwear Knitting Mills,Outerwear Knitting Mills,NaN,NaN,2002,2007


In [29]:
naics_wide[naics_wide['year_min'] == 2012].sample(3)

,NAICS,NAICS_level,NAICS_desc_2002,NAICS_desc_2007,NAICS_desc_2012,NAICS_desc_2017,year_min,year_max
353,311824,6,NaN,NaN,"Dry Pasta, Dough, and Flour Mixes Manufacturin...","Dry Pasta, Dough, and Flour Mixes Manufacturin...",2012,2017
451,315220,6,NaN,NaN,Men’s and Boys’ Cut and Sew Apparel Manufactur...,MenÕs and BoysÕ Cut and Sew Apparel Manufactur...,2012,2017
926,334118,6,NaN,NaN,Computer Terminal and Other Computer Periphera...,Computer Terminal and Other Computer Periphera...,2012,2017


In [30]:
# Select a description - use the 2007 if available, otherwise other
cond = [~naics_wide['NAICS_desc_2007'].isna(),
        ~naics_wide['NAICS_desc_2012'].isna(),
         ~naics_wide['NAICS_desc_2017'].isna()]
choice = [naics_wide['NAICS_desc_2007'], naics_wide['NAICS_desc_2012'],
           naics_wide['NAICS_desc_2017']]
naics_wide['NAICS_desc'] = np.select(cond, choice, naics_wide['NAICS_desc_2002'])

In [31]:
naics_wide['NAICS_desc'].isna().value_counts()

NAICS_desc
False    2466
Name: count, dtype: int64

In [32]:
naics_wide.to_csv('./naics_desc_wide.csv', index=False)
naics_wide.to_parquet('./naics_desc_wide.parquet')

# Create Simple Labeling File for 6 Digit NAICS

#### Get best description
Choose one year per code.

In [33]:
# Get the best description (choose one year)
naics_all_long['rank_yr'] = np.where(naics_all_long['year'] == 2007, 1, -naics_all_long['year'] )
naics_long = naics_all_long.sort_values(['NAICS','rank_yr']) \
    .groupby('NAICS') \
    .head(1) \
    .drop(columns=['rank_yr'])
naics_long.head()

,NAICS,NAICS_desc,year,NAICS_level,NAICS_sector_range
1,11,"Agriculture, Forestry, Fishing and Hunting",2017,2,11
2,111,Crop Production,2017,3,NaN
3,1111,Oilseed and Grain Farming,2017,4,NaN
4,11111,Soybean Farming,2017,5,NaN
5,111110,Soybean Farming,2017,6,NaN


In [34]:
naics_long.to_csv('./naics_desc_long.csv', index=False)
naics_long.to_parquet('./naics_desc_long.parquet')

In [35]:
naics_long.shape

(2466, 5)

##### Get all level descriptions for 6 digit codes

In [36]:
def get_hier_label(naics_6, niacs_hier, level=None):
    
    if level is None:
        level = len(niacs_hier['NAICS'].iloc[0])

    if level == 2:
        label = 'NAICS_sector' 
    else:
        label = 'NAICS_' + str(level) 
    
    naics_6 = naics_6.copy()
    naics_6['other'] = naics_6['NAICS'].str[0:level]

    merge = naics_6.merge(niacs_hier[['NAICS', 'NAICS_desc']] \
            .rename(columns={'NAICS': 'other',
                             'NAICS_desc':label + '_desc'}),
               on='other', how='left') \
        .drop(columns=['other', 'NAICS_level'], errors='ignore')
    return merge

In [37]:
naics_6 = naics_long[naics_long['NAICS_level'] == 6] \
    .drop(columns=['year', 'NAICS_sector_range'], errors='ignore')
naics_hier = naics_long[naics_long['NAICS_level'] != 6]

In [38]:
naics_6.shape

(1283, 3)

In [39]:
naics_long_desc = naics_6.drop(columns=['year', 'rank_yr'], errors='ignore')
for name, group in naics_hier.groupby('NAICS_level'):
    naics_long_desc = get_hier_label(naics_long_desc, group,
                                     name)
naics_long_desc.sample(3)

,NAICS,NAICS_desc,NAICS_sector_desc,NAICS_3_desc,NAICS_4_desc,NAICS_5_desc
1223,811490,Other Personal and Household Goods Repair and ...,Other Services (except Public Administration),Repair and Maintenance,Personal and Household Goods Repair and Mainte...,Other Personal and Household Goods Repair and ...
314,323112,Commercial Flexographic Printing,Manufacturing,Printing and Related Support Activities,Printing and Related Support Activities,Printing
1222,811430,Footwear and Leather Goods Repair,Other Services (except Public Administration),Repair and Maintenance,Personal and Household Goods Repair and Mainte...,Footwear and Leather Goods Repair


In [40]:
naics_long_desc['NAICS_sector_desc'].value_counts(dropna=False)

NAICS_sector_desc
Manufacturing                                                               538
Retail Trade                                                                 83
Wholesale Trade                                                              71
Agriculture, Forestry, Fishing and Hunting                                   64
Transportation and Warehousing                                               57
Professional, Scientific, and Technical Services                             52
Other Services (except Public Administration)                                49
Administrative and Support and Waste Management and Remediation Services     45
Information                                                                  43
Finance and Insurance                                                        42
Health Care and Social Assistance                                            39
Mining, Quarrying, and Oil and Gas Extraction                                32
Construction          

In [41]:
naics_long_desc.to_csv('./naics_desc_long_all_labels.csv', index=False)
naics_long_desc.to_parquet('./naics_desc_long_all_labels.parquet')

In [52]:
naics_long_desc['NAICS_sector_desc'].isna().value_counts()

NAICS_sector_desc
False    1283
Name: count, dtype: int64

##### Create 4-digit NAICS map

In [42]:
naics_4 = naics_long[naics_long['NAICS_level'] == 4] \
    .drop(columns=['year', 'NAICS_sector_range'], errors='ignore')
naics_hier_4 = naics_long[naics_long['NAICS_level'] < 4]

In [55]:
naics_4_long_desc = naics_4.drop(columns=['year', 'rank_yr'], errors='ignore')
for name, group in naics_hier_4.groupby('NAICS_level'):
    naics_4_long_desc = get_hier_label(naics_4_long_desc, group,
                                     name)
naics_4_long_desc.rename(columns={'NAICS':'NAICS_4',
                                 'NAICS_desc':'NAICS_4_desc'}, inplace=True)
naics_4_long_desc.sample(3)

,NAICS_4,NAICS_4_desc,NAICS_sector_desc,NAICS_3_desc
136,4245,Farm Product Raw Material Merchant Wholesalers,Wholesale Trade,"Merchant Wholesalers, Nondurable Goods"
61,3219,Other Wood Product Manufacturing,Manufacturing,Wood Product Manufacturing
81,3312,Steel Product Manufacturing from Purchased Steel,Manufacturing,Primary Metal Manufacturing


In [47]:
naics_4_long_desc.to_csv('./naics4_long_all_labels.csv', index=False)
naics_4_long_desc.to_parquet('./naics4_long_all_labels.parquet')

##### Create 3-digit NAICS map

In [56]:
naics_3 = naics_long[naics_long['NAICS_level'] == 3] \
    .drop(columns=['year', 'NAICS_sector_range'], errors='ignore')
naics_hier_3 = naics_long[naics_long['NAICS_level'] < 3]
naics_3_long_desc = naics_3.drop(columns=['year', 'rank_yr'], errors='ignore')
for name, group in naics_hier_3.groupby('NAICS_level'):
    naics_3_long_desc = get_hier_label(naics_3_long_desc, group,
                                     name)
naics_3_long_desc.rename(columns={'NAICS':'NAICS_3',
                                 'NAICS_desc':'NAICS_3_desc'}, inplace=True)
naics_3_long_desc.sample(3)

,NAICS_3,NAICS_3_desc,NAICS_sector_desc
21,324,Petroleum and Coal Products Manufacturing,Manufacturing
54,487,Scenic and Sightseeing Transportation,Transportation and Warehousing
92,921,"Executive, Legislative, and Other General Gove...",Public Administration


In [57]:
naics_3_long_desc.to_csv('./naics3_long_all_labels.csv', index=False)
naics_3_long_desc.to_parquet('./naics3_long_all_labels.parquet')

##### Finally the 2 digit map

In [50]:
naics_2_long_desc = naics_long[naics_long['NAICS_level'] == 2] \
    .drop(columns=['NAICS_level', 'NAICS_sector_range', 'year']) \
    .rename(columns={'NAICS':'NAICS_2',
                    'NAICS_desc':'NAICS_sector_desc'})
naics_2_long_desc

,NAICS_2,NAICS_sector_desc
1,11,"Agriculture, Forestry, Fishing and Hunting"
132,21,"Mining, Quarrying, and Oil and Gas Extraction"
180,22,Utilities
205,23,Construction
278,31,Manufacturing
278,32,Manufacturing
278,33,Manufacturing
926,42,Wholesale Trade
1091,44,Retail Trade
1091,45,Retail Trade


In [51]:
naics_2_long_desc.to_csv('./naics2_long_all_labels.csv', index=False)
naics_2_long_desc.to_parquet('./naics2_long_all_labels.parquet')